## Implement Collaborative Filtering from Scratch

This notebook explores fast collaborative filtering algorithm by employing a vectorized implementation, no for loops allowed :). References and inspirations from USF ML course by Yannet Interian and <a href=http://www.fast.ai/>Fast.ai</a> by Jeremy Howard.

### Open Form Formulas for Updates

**Basic Notation**

* $U$: matrix of u x n (u: # of users, n: dim of embedding)
* $V$: matrix of v x n (v: # of items, n: dim of embedding)
* $u^{0}$: vector of u x 1 bias of users
* $v^{0}$: vector of v x 1 bias of movies
* $n_{u}$: number of users
* $n_{i}$: number of items(movie)


Total # of parameters to be learned $n_{u}\ x\ (D + 1)\ + n_{i}\ x\ (D + 1)\ =\ (n_{u}\ +\ n_{v}) x\ (D + 1)$

**Derivatives with biases**

i) $\frac{\partial E}{\partial u_{ik}} = 
\frac{2}{N}\sum_{(i,j);r_{ij}=1}{
(u_{0i} + v_{0j} + u_{i}v_{j} - y_{ij})}v_{jk} $

ii) $\frac{\partial E}{\partial v_{jk}} = 
\frac{2}{N}\sum_{(i,j);r_{ij}=1}{
(u_{0i} + v_{0j} + u_{i}v_{j} - y_{ij})}u_{ik} $

iii)$\frac{\partial E}{\partial u_{0k}} = 
\frac{2}{N}\sum_{(j);r_{ij}=1}\sum_{(i,j);r_{ij}=1}{
(u_{0i} + v_{0j} + u_{i}v_{j} - y_{ij})}$

iv)$\frac{\partial E}{\partial v_{0k}} = 
\frac{2}{N}\sum_{(i);r_{ij}=1}\sum_{(i,j);r_{ij}=1}{
(u_{0i} + v_{0j} + u_{i}v_{j} - y_{ij})}$

**Gradient Descent Updates**

i) $u_{ik} = u_{ik} - \eta{\frac{\partial E}{\partial u_{ik}}}$

ii) $v_{jk} = v_{jk} - \eta{\frac{\partial E}{\partial v_{jk}}}$

iii) $u_{0k} = u_{0k} - \eta{\frac{\partial E}{\partial u_{0k}}}$

iv) $v_{0k} = v_{0k} - \eta{\frac{\partial E}{\partial v_{0k}}}$

### Vectorized Form Formulas for Updates

**Vectorized Derivatives with Biases**

Let's define our notations:

$n_{u}$: number of users

$n_{i}$: number of items(movie)

$U = n_{u}\ x\ n_{D}$ user matrix

$V = n_{i}\ x\ n_{D}$ item(movie) matrix

$Y = n_{u}\ x\ n_{i}$ ground truth of ratings sparse

$R = n_{u}\ x\ n_{i}$ binary matrix indicating ratings sparse

$M = n_{u}\ x\ 1$ column vector of ones

$N = n_{i}\ x\ 1$ column vector of ones

$B_{u} = n_{u}\ x\ n_{i}$ matrix from broadcasted column vector of user biases $b_{u}$

$B_{i} = n_{i}\ x\ n_{u}$ matrix from broadcasted column vector of item biases $b_{i}$


i) $\frac{\partial E}{\partial U}: -\frac{2}{N}
    ((Y - UV^{T} - B_{u} - B_{i}^{T})*R)V$
    
ii)$\frac{\partial E}{\partial V}: -\frac{2}{N}
    ((Y - UV^{T} - B_{u} - B_{i}^{T})*R)^{T}U$
    
iii)$\frac{\partial E}{\partial b_{u}}: -\frac{2}{N}
    ((Y - UV^{T} - B_{u} - B_{i}^{T})*R)N$   

iii)$\frac{\partial E}{\partial b_{i}}: -\frac{2}{N}
    ((Y - UV^{T} - B_{u} - B_{i}^{T})*R)^{T}M$   
    
    
**Adding Regularization**

Let's define our new loss:


$\sum:$ sums all elements of matrix

$*:$ element wise multiplication

Loss = $\sum(Y - UV^{T} - B_{u} - B_{i}^{T})*R)^{2}/N + \lambda\sum U*U + \lambda\sum V*V $

i) $\frac{\partial E}{\partial U}: -\frac{2}{N}
    ((Y - UV^{T} - B_{u} - B_{i}^{T})*R)V  + 2\lambda U$
    
ii)$\frac{\partial E}{\partial V}: -\frac{2}{N}
    ((Y - UV^{T} - B_{u} - B_{i}^{T})*R)^{T}U + 2\lambda V$
    
iii)$\frac{\partial E}{\partial b_{u}}: -\frac{2}{N}
    ((Y - UV^{T} - B_{u} - B_{i}^{T})*R)N $   

iii)$\frac{\partial E}{\partial b_{i}}: -\frac{2}{N}
    ((Y - UV^{T} - B_{u} - B_{i}^{T})*R)^{T}M$   
    

### Imports

In [14]:
import numpy as np
import pandas as pd
from scipy import sparse

### Functions

In [15]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [16]:
# we will create indices to be matched by our embeddings later
def encode_data(df):
    new_df = df.copy()
    _, user_col, num_users = proc_col(df["userId"])
    _, movies_col, num_movies = proc_col(df["movieId"])
    new_df["userId"], new_df["movieId"] = user_col, movies_col
    return new_df, num_users, num_movies

In [20]:
# Encode a given target dataframe with same mapping as the source dataframe
# source: df_train, target: df_val
def encode_new_data(df_val, df_train):
    # YOUR CODE HERE
    val = df_val.copy()
    user2idx, _, _ = proc_col(df_train["userId"])
    movie2idx, _, _ = proc_col(df_train["movieId"])
    
    val["userId"] = np.array([user2idx[x] if x in user2idx else np.nan for x in val.userId])
    val["movieId"] = np.array([movie2idx[x] if x in movie2idx else np.nan for x in val.movieId])
        
    # drop missing
    val.dropna(inplace=True)
    val["userId"] = val["userId"].astype('int')
    val["movieId"] = val["movieId"].astype('int')
    return val

In [17]:
# create sparse matrix for fast calculations
def df2matrix(df, nrows, ncols, column_name="rating"):
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [18]:
# This function will create embeddings
# with given dimension K having n unique indices
def create_embedings(n, K):
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

In [6]:
# This function will make predictions for us
# A single prediction for ith user and jth item 
# is dot product of user i's embedding vector and item j's embedding vector
# plus bias term of user i and bias term of item j

def sparse_multiply(df, emb_user, emb_movie, bias_user, bias_movie):
    df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)+\
        bias_user[df["userId"].values].flatten() + bias_movie[df["movieId"].values].flatten()
    return df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="Prediction")

In [39]:
# We will define error to be simple mse without taking account bias terms
# Any desired cost function can be used here
# But optimization will be based on regularized mse loss

def cost(df, emb_user, emb_movie, bias_user, bias_movie):
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    error = np.sum((Y - sparse_multiply(df, emb_user, emb_movie,bias_user, bias_movie)).power(2)) / Y.count_nonzero()
    return error

In [40]:
# Gradient update function which returns
# Gradients for user, movie embedding matrices and bias vectors

def gradient(df, Y, emb_user, emb_movie, bias_user, bias_movie, reg_lambda):
    N = Y.count_nonzero()
    Y_predict= sparse_multiply(df, emb_user, emb_movie, bias_user, bias_movie)
    delta = (Y - Y_predict).toarray()
    
    # compute user/movie grad
    d_emb_user = -(2/N)*np.dot(delta, emb_movie) + 2*reg_lambda*emb_user
    d_emb_movie = -(2/N)*np.dot(delta.T, emb_user) + 2*reg_lambda*emb_movie
    d_bias_user = -(2/N)*np.sum(delta, 1)[:, None]
    d_bias_movie = -(2/N)*np.sum(delta.T, 1)[:, None]
    
    return d_emb_user, d_emb_movie, d_bias_user, d_bias_movie

In [41]:
# This is an implementation of SGD with momentum
# This function will allow us to optimize our parameters
# Based on regularied mse loss

def gradient_descent(df, emb_user, emb_movie, bias_user, bias_movie, reg_lambda=0,\
                                                     iterations=100, learning_rate=0.01, rho=0.9, df_val=None):

    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    v_user_prev, v_movie_prev, v_user_bias_prev, v_movie_bias_prev =  gradient(df, Y, emb_user, emb_movie, \
                                                                      bias_user, bias_movie, reg_lambda)
    for i in range(iterations):
        d_emb_user, d_emb_movie, d_bias_user, d_bias_movie = gradient(df, Y, emb_user, emb_movie, \
                                                                      bias_user, bias_movie, reg_lambda)
        
        v_user = (rho*v_user_prev) + (1-rho)*d_emb_user
        v_movie = (rho*v_movie_prev) + (1-rho)*d_emb_movie
        v_bias_user = (rho*v_user_bias_prev) + (1-rho)*d_bias_user
        v_bias_movie = (rho*v_movie_bias_prev) + (1-rho)*d_bias_movie
        
        
        emb_user -= learning_rate*v_user
        emb_movie -= learning_rate*v_movie
        bias_user -= learning_rate*v_bias_user
        bias_movie -= learning_rate*v_bias_movie
        
        
        
        
        v_user_prev, v_movie_prev, v_user_bias_prev, v_movie_bias_prev =\
                        v_user, v_movie, v_bias_user, v_bias_movie
        
        #print cost every 50th iter
        if (i+1) % 250 == 0:
            train_loss = cost(df, emb_user, emb_movie, bias_user, bias_movie)
            if df_val is not None:
                val_loss = cost(df_val, emb_user, emb_movie, bias_user, bias_movie)
                print(f'Step {i+1} [{train_loss}, {val_loss}]')
            else:
                print(f'Step {i+1} [{train_loss}]')
            
    return emb_user, emb_movie, bias_user, bias_movie

### Training with Sample Movielens

Data can be downloaded here: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

This is a small subset of movielens data ratings

In [53]:
# Don't change this path use a simlink if you have the data somewhere else
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
# sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
# at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train)
df_val = encode_new_data(val, train)
print(len(val), len(df_val), df_train.shape)

20205 19507 (79799, 4)


In [54]:
# K - dimension of embeddings is a hyperparameter
K = 30
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
bias_user = create_embedings(num_users, 1)
bias_movie = create_embedings(num_movies, 1)

In [55]:
%%time
# without regularization for 5000 epochs
# It overfits while converging
emb_user, emb_movie, bias_user, bias_movie = gradient_descent(df_train, emb_user, emb_movie, bias_user, bias_movie,\
                                   1, iterations=5000, learning_rate=2.5, df_val=df_val)

Step 250 [2.4328130017062644, 2.4470552982085603]
Step 500 [1.7557838010353581, 1.770211858161878]
Step 750 [1.4577264112860782, 1.4745802735352518]
Step 1000 [1.2885065924327133, 1.3098002130830877]
Step 1250 [1.1788292788989598, 1.2054203458912132]
Step 1500 [1.1015795915679523, 1.1336880876918307]
Step 1750 [1.0439369147935018, 1.0814823959721538]
Step 2000 [0.9990564922081951, 1.041826535060369]
Step 2250 [0.9629599262417026, 1.0106904577600788]
Step 2500 [0.9331823527015705, 0.9855958292272023]
Step 2750 [0.9081171102812573, 0.9649406510224539]
Step 3000 [0.8866720786469449, 0.9476458658624539]
Step 3250 [0.8680780750518733, 0.9329588925431501]
Step 3500 [0.8517766174081155, 0.9203389029138663]
Step 3750 [0.8373513988701243, 0.9093870155722301]
Step 4000 [0.8244849281180713, 0.8998022911347189]
Step 4250 [0.8129302157699767, 0.8913531199640878]
Step 4500 [0.8024917585150053, 0.8838580997854993]
Step 4750 [0.7930124402191663, 0.877172936899896]
Step 5000 [0.7843642998332703, 0.8711

### Clustering Embeddings

In [68]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [60]:
emb_movie.shape

(8442, 30)

In [62]:
tsne_movie = TSNE(n_components=2).fit_transform(emb_movie)

In [63]:
tsne_movie.shape

(8442, 2)

In [69]:
df_train.head()

,userId,movieId,rating,timestamp,Prediction
0,0,0,2.5,1260759144,2.167647
1,0,1,3.0,1260759179,2.757539
2,0,2,3.0,1260759182,2.541253
3,0,3,2.0,1260759185,2.413623
6,0,4,2.0,1260759187,2.880511


In [70]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

In [90]:
movie2idx, _, _ = proc_col(train['movieId'])

In [84]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [95]:
movies['embidx'] = [movie2idx[i] if i in movie2idx else np.nan for i in movies.movieId ]

In [97]:
movies.dropna(inplace=True)

In [101]:
movies['embidx'] = movies['embidx'].astype('int')

In [102]:
movies.head()

,movieId,title,genres,embidx
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,349
1,2,Jumanji (1995),Adventure|Children|Fantasy,1771
2,3,Grumpier Old Men (1995),Comedy|Romance,268
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1772
4,5,Father of the Bride Part II (1995),Comedy,560


In [123]:
index = np.array(range(tsne_movie.shape[0]))[:, None]
tsne_df = pd.DataFrame(data = np.concatenate([tsne_movie,index], 1), 
             columns=['tsne1', 'tsne2', 'embidx'])

In [124]:
tsne_df.embidx = tsne_df.embidx.astype('int')

In [127]:
tsne_movie_df = pd.merge(tsne_df, movies, how='left', on='embidx')

In [128]:
tsne_movie_df.head()

,tsne1,tsne2,embidx,movieId,title,genres
0,0.657654,-1.022516,0,31,Dangerous Minds (1995),Drama
1,0.009028,2.416170,1,1029,Dumbo (1941),Animation|Children|Drama|Musical
2,-0.461207,-2.352748,2,1061,Sleepers (1996),Thriller
3,2.156852,2.165390,3,1129,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,-0.285314,3.416818,4,1287,Ben-Hur (1959),Action|Adventure|Drama


In [133]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [150]:
data = []
genres = tsne_movie_df.genres.unique()
for genre in genres:
    msk = tsne_movie_df.genres== genre
    trace = go.Scatter(
        x = tsne_movie_df.tsne1[msk],
        y = tsne_movie_df.tsne2[msk],
        mode = 'markers'
    )
    data.append(trace)

In [151]:
py.iplot(data, sharing='public', filename='tsne_movies')